# Stochastic Gradient Descent (SGD) Implementation

## 1. Definition
**Stochastic Gradient Descent (SGD)** is an optimization algorithm used to train machine learning models. Unlike **Batch Gradient Descent**, which calculates the error for the *entire* dataset before updating weights, SGD calculates the error and updates the weights for **one single training example** at a time.

## 2. The Need for SGD
Why do we use SGD when Batch Gradient Descent is mathematically more stable?
* **Speed:** When the dataset has millions of rows ($n$), calculating the gradient for the whole dataset is computationally expensive and slow. SGD updates weights instantly after looking at one row.
* **Memory:** Batch GD requires loading the whole dataset into memory to calculate gradients. SGD only needs one row in memory at a time.
* **Escaping Local Minima:** The "noisy" updates of SGD (jumping around) can sometimes help the model escape shallow local minima that might trap Batch GD.

## 3. Implementation Steps
1.  **Initialize** weights and intercept arbitrarily.
2.  **Loop** for a fixed number of epochs.
3.  **Random Sampling:** inside the loop, select **one random index** from the dataset.
4.  **Forward Pass:** Calculate prediction $\hat{y}$ for that single row.
5.  **Gradient Calculation:**
    $$\text{Gradient} = -2 \cdot (y_i - \hat{y}_i) \cdot x_i$$
6.  **Update Weights:**
    $$w_{new} = w_{old} - \text{learning\_rate} \cdot \text{Gradient}$$

In [1]:
from sklearn.datasets import load_diabetes

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import time

In [2]:
X,y = load_diabetes(return_X_y=True)

In [3]:
print(X.shape)
print(y.shape)

(442, 10)
(442,)


In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [5]:
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [6]:
print(reg.coef_)
print(reg.intercept_)

[  -9.15865318 -205.45432163  516.69374454  340.61999905 -895.5520019
  561.22067904  153.89310954  126.73139688  861.12700152   52.42112238]
151.88331005254167


In [7]:
y_pred = reg.predict(X_test)
r2_score(y_test,y_pred)

0.4399338661568968

## 4. Custom SGD Regressor Class
Here we implement the algorithm.
* **Logic:** We iterate through the dataset, but instead of using the data sequentially, we pick a **random index** (`idx`) at every step. This randomness makes it "Stochastic".
* **Vectorization:** Notice that we do not use matrix multiplication for the whole dataset; we only apply dot products to the single selected row.

In [8]:
class SGDRegressor:
    
    def __init__(self,learning_rate=0.01,epochs=100):
        
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        
    def fit(self,X_train,y_train):
        # init your coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        
        for i in range(self.epochs):
            for j in range(X_train.shape[0]):
                idx = np.random.randint(0,X_train.shape[0])
                
                y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_
                
                intercept_der = -2 * (y_train[idx] - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_der)
                
                coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
                self.coef_ = self.coef_ - (self.lr * coef_der)
        
        print(self.intercept_,self.coef_)
    
    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

In [9]:
sgd = SGDRegressor(learning_rate=0.01,epochs=40)

In [10]:
start = time.time()
sgd.fit(X_train,y_train)
print("The time taken is",time.time() - start)

144.58089475555818 [  69.33611626  -51.8839702   319.02414835  227.55569791   29.52275956
  -10.72355504 -164.26467449  130.36661757  294.8521573   128.13487899]
The time taken is 0.5330750942230225


In [11]:
y_pred = sgd.predict(X_test)

### Results Analysis
* **Time Taken:** SGD is generally very fast per iteration.
* **Accuracy:** The $R^2$ score is slightly lower than the OLS baseline. This is expected because SGD is an approximation algorithm that bounces around the minimum rather than settling perfectly at the bottom.

In [12]:
r2_score(y_test,y_pred)

0.4111457132351044

## 5. Comparison with Scikit-Learn
Now we check how our scratch implementation compares to the optimized `SGDRegressor` from the Scikit-Learn library.

In [13]:
from sklearn.linear_model import SGDRegressor

In [14]:
reg = SGDRegressor(max_iter=100,learning_rate='constant',eta0=0.01)

In [15]:
reg.fit(X_train,y_train)

C:\Users\Rajdeep Ramola\anaconda3\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDRegressor(learning_rate='constant', max_iter=100)

In [16]:
y_pred = reg.predict(X_test)

In [17]:
r2_score(y_test,y_pred)

0.43104344281830187